### Importing Libraries

In [1]:
import csv
from utils import *
import pandas as pd
import numpy as np

### Establishing Connection with cursor

In [2]:
conn_db = make_connection(config_file = 'config_files/db_superstore.ini')
cursor_db = conn_db.cursor()

conn_wh = make_connection(config_file = 'config_files/wh_superstore.ini')
cursor_wh = conn_wh.cursor()

### Drop all existing tables (if any)

In [3]:
def drop(cursor, tables = ['sales_fact','listed_fact','website_dim','calender_dim','product_dim', 'customer_dim']): 

    for table in tables:
        cursor.execute(f"DROP TABLE IF EXISTS {table}")

In [4]:
drop(cursor_wh)

### Create and Insert dimension tables

`customer_dim`

In [5]:
sql = ( """
        CREATE TABLE customer_dim
        (
          customerkey INT NOT NULL AUTO_INCREMENT,
          customer_id INT NOT NULL,
          customer_first_name CHAR(30) NOT NULL,
          customer_last_name CHAR(30) NOT NULL,
          street_address CHAR(40) NOT NULL,
          city CHAR(30) NOT NULL,
          state CHAR(30) NOT NULL,
          country CHAR(30) NOT NULL,
          postal_code INT NOT NULL,
          segment CHAR(15) NOT NULL,
          customer_rating INT,
          PRIMARY KEY (customerkey)
        );
        """
      )

make_table('customer_dim', sql, cursor_wh)

In [6]:
sql = ( """
        INSERT INTO datapirates_wh.customer_dim(customer_id, customer_first_name, customer_last_name, street_address,city,
        state, country, postal_code, segment, customer_rating)
        SELECT customer.customer_id, customer_first_name, customer_last_name, street_address, city, state, country, postal_code,
        segment, AVG(customer_rating)
        FROM customer LEFT JOIN feedback ON customer.customer_id = feedback.customer_id
        GROUP BY customer.customer_id
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

display_table('customer_dim', conn_wh, rows = 1)

,customerkey,customer_id,customer_first_name,customer_last_name,street_address,city,state,country,postal_code,segment,customer_rating
0,1,100000,Cazzie,Crimin,Sapshed,Annapolis,Maryland,United States,21405,Home Office,NaN
1,2,100001,Lowe,Greguoli,Kynder,Dallas,Texas,United States,75205,Corporate,NaN
2,3,100002,Mil,Gansbuhler,Winchester,Fresno,California,United States,93715,Home Office,1.0
3,4,100003,Sarena,Suston,Abbis,New York City,New York,United States,10280,Consumer,4.0
4,5,100004,Gavan,Desaur,Kop,Huntington,West Virginia,United States,25770,Home Office,NaN


`product_dim`

In [7]:
sql = ( """
        CREATE TABLE product_dim
        (
          productkey INT NOT NULL AUTO_INCREMENT,
          product_id VARCHAR(15) NOT NULL,
          product_name VARCHAR(255) NOT NULL,
          base_price FLOAT NOT NULL,
          quantity_available INT NOT NULL,
          category_name CHAR(20) NOT NULL,
          distributor_name CHAR(30) NOT NULL,
          distributor_email VARCHAR(40) NOT NULL,
          region CHAR(10) NOT NULL,
          PRIMARY KEY (productkey)
        );
        """
      )

make_table('product_dim', sql, cursor_wh)

In [8]:
sql = ( """
        INSERT INTO datapirates_wh.product_dim(product_id, 
                                           product_name,
                                           base_price,
                                           quantity_available,
                                           category_name,
                                           distributor_name, distributor_email, region)
            SELECT product_id, product_name, base_price, 
                   quantity_available, category_name, distributor_name, distributor_email, region
            FROM product , category, distributor
            WHERE distributor.distributor_id = product.distributor_id
            AND product.category_id = category.category_id
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

display_table('product_dim', conn_wh, rows = 1)

,productkey,product_id,product_name,base_price,quantity_available,category_name,distributor_name,distributor_email,region
0,1,P1000001,3M Hangers With Command Adhesive,2.96,120,Furniture,Roys Distributing,info@roysdistributing.com,South
1,2,P1000002,Eldon 100 Class Desk Accessories,4.04,60,Furniture,ABC Warehouse,abc@warehouse.com,West
2,3,P1000003,Regeneration Desk Collection,5.28,120,Furniture,District Wholesale,fred@districtwholesale.com,West
3,4,P1000004,Longer-Life Soft White Bulbs,6.16,60,Furniture,Middle Man Distributors,orders@middleman.com,Central
4,5,P1000005,"6"" Cubicle Wall Clock, Black",9.71,90,Furniture,Wholesale Solutions LLC,wholesalesolutions@gmail.com,Northweast


`calender_dim`

In [9]:
sql = ( """
        CREATE TABLE calendar_dim
        (
            calendarkey INT NOT NULL AUTO_INCREMENT,
            full_date DATE,
            day_of_week VARCHAR(9),
            date INT,
            month_number INT,
            month_name VARCHAR(10),
            qtr INT,
            year INT,
            PRIMARY KEY (calendarkey)
        )
        """
      )

make_table('calendar_dim', sql, cursor_wh)

In [10]:
sql = ( """
        INSERT INTO datapirates_wh.calendar_dim(full_date, day_of_week, 
                                            date, month_number, month_name, 
                                            qtr, year)
            SELECT DISTINCT order_date, DAYNAME(order_date), 
                            DAY(order_date), MONTH(order_date), MONTHNAME(order_date), 
                            QUARTER(order_date), YEAR(order_date)
            FROM orders
        """
      )

cursor_db.execute(sql)
conn_db.commit()

display_table('calendar_dim', conn_wh, rows = 1)

,calendarkey,full_date,day_of_week,date,month_number,month_name,qtr,year
0,1,2022-01-01,Saturday,1,1,January,1,2022
1,2,2022-01-02,Sunday,2,1,January,1,2022
2,3,2022-01-03,Monday,3,1,January,1,2022
3,4,2022-01-04,Tuesday,4,1,January,1,2022
4,5,2022-01-05,Wednesday,5,1,January,1,2022


`website_dim`

In [11]:
sql = ( """
        CREATE TABLE website_dim
        (
            websitekey INT NOT NULL AUTO_INCREMENT,
            website_name CHAR(20) NOT NULL,
            listing_fee FLOAT NOT NULL,
            PRIMARY KEY (websitekey)
        );
        """
      )

make_table('website_dim', sql, cursor_wh)

In [12]:
sql = ( """
        INSERT INTO datapirates_wh.website_dim(website_name, listing_fee)
            SELECT website_name, listing_fee
            FROM website
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

display_table('website_dim', conn_wh, rows = 1)

,websitekey,website_name,listing_fee
0,1,Amazon,0.30
1,2,Costco,0.20
2,3,Ikea,0.25
3,4,Target,0.50
4,5,Walmart,0.50


### Create and Insert fact tables

`listed_fact`

In [13]:
sql = ( """
        CREATE TABLE listed_fact
        (
            websitekey INT NOT NULL,
            productkey INT NOT NULL,
            price FLOAT NOT NULL,
            PRIMARY KEY (websitekey, productkey),
            FOREIGN KEY (websitekey) REFERENCES website_dim(websitekey),
            FOREIGN KEY (productkey) REFERENCES product_dim(productkey)
        );
        """
      )

make_table('listed_fac', sql, cursor_wh)

In [14]:
sql = ( """
        INSERT INTO datapirates_wh.listed_fact(websitekey, productkey, price)
            SELECT datapirates_wh.website_dim.websitekey, datapirates_wh.product_dim.productkey, listed.price
            FROM listed, datapirates_wh.website_dim, datapirates_wh.product_dim, product, website
            WHERE listed.product_id = product.product_id
            AND product.product_id = datapirates_wh.product_dim.product_id
            AND listed.website_name = website.website_name
            AND website.website_name = datapirates_wh.website_dim.website_name
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

display_table('listed_fact', conn_wh, rows = 1)

,websitekey,productkey,price
0,1,1,3.26
1,1,2,4.34
2,1,3,5.58
3,1,4,6.46
4,1,5,10.01


`sales_fact`

In [15]:
sql = ( """
        CREATE TABLE sales_fact
        (
            customerkey INT NOT NULL,
            calendarkey INT NOT NULL,
            websitekey INT NOT NULL,
            productkey INT NOT NULL,
            order_id INT NOT NULL,
            sales FLOAT,
            units_sold INT,
            PRIMARY KEY (productkey, order_id, websitekey),
            FOREIGN KEY (customerkey) REFERENCES customer_dim(customerkey),
            FOREIGN KEY (calendarkey) REFERENCES calendar_dim(calendarkey),
            FOREIGN KEY (websitekey) REFERENCES website_dim(websitekey),
            FOREIGN KEY (productkey) REFERENCES product_dim(productkey)
        );
        """
      )

make_table('sales_fact', sql, cursor_wh)

In [16]:
sql = ( """
        INSERT INTO datapirates_wh.sales_fact(customerkey, calendarkey, websitekey, productkey, order_id, sales, 
        units_sold)
            SELECT datapirates_wh.customer_dim.customerkey, datapirates_wh.calendar_dim.calendarkey, 
            datapirates_wh.website_dim.websitekey, datapirates_wh.product_dim.productkey, orders.order_id, 
            orders.order_total, shopping_cart.quantity_ordered
            FROM datapirates_wh.customer_dim, datapirates_wh.calendar_dim, datapirates_wh.website_dim, 
            datapirates_wh.product_dim, orders, shopping_cart
            WHERE orders.order_id = shopping_cart.order_id
            AND orders.order_date = datapirates_wh.calendar_dim.full_date
            AND orders.customer_id = datapirates_wh.customer_dim.customer_id
            AND shopping_cart.product_id = datapirates_wh.product_dim.product_id
            AND shopping_cart.website_name = datapirates_wh.website_dim.website_name
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

display_table('sales_fact', conn_wh, rows = 1)

,customerkey,calendarkey,websitekey,productkey,order_id,sales,units_sold
0,996,5,3,1,10000024,465.84,6
1,25,12,1,1,10000067,28.56,6
2,194,62,3,1,10000350,192.92,4
3,113,73,3,1,10000416,39.31,11
4,503,89,1,1,10000511,18.78,3


### Closing the cursor

In [17]:
cursor_db.close()
conn_db.close()

cursor_wh.close()
conn_wh.close()